In [16]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 20 20:25:30 2016

@author: Noklam
"""

## This Project is greatly inspired by the discussion, which help me a lot to 
## debug and I have give up my original code and follow their structure and 
## made my own classifier 
## Especially helpful discussion from vivek_29420285151271 and michael_807478
## As I am not a pro-programmer, doesnt develop habit of writing nice function
## Their well-written function have helped me a lot to simplify the final 
## version
######Steps:
# 1. Remove outlier "TOTAL"
# 2. Remove unwanted features
# 3. Log transformation
# 4. Pipeline
# 5. Imputer 0 for NaN, MinMaxScaler
# 6. Select K-best , K=8
# 7. PCA ,choose first 4 
# 8. Run Classifier AdaBoostClassifier,
# 9. Precision =0.4 ,Recall =0.33
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 20 20:25:30 2016

@author: Noklam
"""
import sys
import pickle
sys.path.append("../tools/")
import matplotlib.pyplot as plt
## allow graph to be displayed
import pylab
import pandas as pd
import numpy as np
from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data
from sklearn.preprocessing import Imputer                                            
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn import cross_validation
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from tester import test_classifier
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import make_scorer
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm

def data_preprocessing(data):
    
   
    with open(data, "r") as data_file:
        data_dict = pickle.load(data_file)
    df = pd.DataFrame.from_dict(data_dict, orient='index', dtype=np.float)
    df = df.drop('TOTAL') ## remove 

    ### Drop all the variable that I have done before, this function is aim to repeat the above process in a nice function.
    df = df.drop('email_address', axis=1)
    df = df.astype(float)
    drop_list=['deferral_payments', 'deferred_income', 'director_fees', 'loan_advances', 'restricted_stock_deferred']
    df=df.drop(drop_list, axis=1)
    ###new variable added
    df['to_ratio'] = df['from_poi_to_this_person']/(df['from_messages'] )
    df['from_ratio'] = df['from_this_person_to_poi']/(df['from_messages'])
    df=df.drop('from_this_person_to_poi',axis=1)
    df=df.drop('from_poi_to_this_person',axis=1)
    df=df.drop('from_messages',axis=1)
    df=df.drop('to_messages',axis=1)
    
    labels=df['poi']
    df=df.drop('poi',axis=1)
    df.replace(to_replace='NaN', value=np.nan, inplace=True)
    df=np.log(df+1)
    features_list=list(df.columns.values)
    
    df.insert(0,'poi',labels)
    df=df.fillna(0)
    features=df[features_list]
    features_list.insert(0,'poi')
    
   

    
    return data_dict,df, features, labels, features_list
def get_train_test_split(features, labels):
    '''This gets the train test split for the sklearn runs of the model'''
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(features, labels, test_size=0.3, random_state=42)
    return features_train, features_test, labels_train, labels_test

def custom_scorer(labels, predictions):
    precision = precision_score(labels,predictions)
    recall = recall_score(labels,predictions )
    min_score = min(precision,recall)
    return min_score
    
scorer  = make_scorer(custom_scorer, greater_is_better=True)

data="final_project_dataset.pkl"
data_dict,df, features, labels, features_list = data_preprocessing(data)

features_train, features_test, labels_train, labels_test=get_train_test_split( features,labels)

## I choose AdaboostClassifier to be my final choice as it seems to provide the highest performance
## and I have tune the parameter which perform best in the training sample
Pipeline_final = Pipeline([
        ('std', MinMaxScaler()),
        ('selection', SelectKBest()),
        ('pca', PCA()),
        ('clf', AdaBoostClassifier(DecisionTreeClassifier(max_depth=5),random_state=0))
    ])
# estimator parameters
    #'
k = [8]
c = [4]
e = [100]
r = [0.001]


param_grid={'selection__k': k,
            'pca__n_components': c,
            'clf__n_estimators': e,
            'clf__learning_rate': r,
             }

# The imputer is useless as I end up fill all NaN with 0 which give me higher accuracy          
# set model parameters to grid search object
gridCV_object_final = GridSearchCV(estimator = Pipeline_final, 
                             param_grid = param_grid,
                             scoring = scorer,
                             cv = StratifiedShuffleSplit(labels_train,200,train_size=0.1,random_state=12))

# train the model
gridCV_object_final.fit(features_train, labels_train)

print gridCV_object_final.best_params_
print gridCV_object_final.scorer_
pred_CV = gridCV_object_final.predict(features_test)
from sklearn.metrics import classification_report
print classification_report(labels_test, pred_CV)
            
from  time import time

df1 = df.transpose()
df1 = df1.to_dict()
print "\n\nAnd these are the results going through the test classifier:\n"
## send the best parameter to clf
t0=time()
clf=gridCV_object_final.best_estimator_
test_classifier(clf, df1, features_list, folds = 1000)
t1=time()-t0
print t1


CLF_PICKLE_FILENAME = "my_classifier.pkl"
DATASET_PICKLE_FILENAME = "my_dataset.pkl"
FEATURE_LIST_FILENAME = "my_feature_list.pkl"
dump_classifier_and_data(clf,df1,features_list)



{'pca__n_components': 4, 'selection__k': 8, 'clf__learning_rate': 0.001, 'clf__n_estimators': 100}
make_scorer(custom_scorer)
             precision    recall  f1-score   support

        0.0       0.97      0.93      0.95        40
        1.0       0.50      0.75      0.60         4

avg / total       0.93      0.91      0.92        44



And these are the results going through the test classifier:

Pipeline(steps=[('std', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selection', SelectKBest(k=8, score_func=<function f_classif at 0x0000000009F909E8>)), ('pca', PCA(copy=True, n_components=4, whiten=False)), ('clf', AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifie...dom_state=None, splitter='best'),
          learning_rate=0.001, n_estimators=100, random_state=0))])
	Accuracy: 0.84527	Precision: 0.40243	Recall: 0.33100	F1: 0.36324	F2: 0.34318
	Total predictions: 15000	True positives:  662	False positives:  983	False negatives: 1338	True negativ

In [2]:
features.head()

,bonus,exercised_stock_options,expenses,long_term_incentive,other,restricted_stock,salary,shared_receipt_with_poi,total_payments,total_stock_value,to_ratio,from_ratio
ALLEN PHILLIP K,15.244625,14.363367,9.537411,12.627431,5.030438,11.744259,12.215805,7.249926,15.316125,14.363367,0.021186,0.029183
BADUM JAMES P,0.000000,12.460009,8.156797,0.000000,0.000000,0.000000,0.000000,0.000000,12.114325,12.460009,0.000000,0.000000
BANNANTINE JAMES M,0.000000,15.213278,10.938485,0.000000,13.669934,14.379433,6.169611,6.144186,13.727988,15.472497,0.852212,0.000000
BAXTER JOHN C,13.997833,15.714710,9.323758,14.276761,14.793951,15.187380,12.495390,0.000000,15.544391,16.178556,0.000000,0.000000
BAY FRANKLIN R,12.899222,0.000000,11.768676,0.000000,4.248495,11.889971,12.387027,0.000000,13.626402,11.051128,0.000000,0.000000


In [3]:

features_list

['poi',
 'bonus',
 'exercised_stock_options',
 'expenses',
 'long_term_incentive',
 'other',
 'restricted_stock',
 'salary',
 'shared_receipt_with_poi',
 'total_payments',
 'total_stock_value',
 'to_ratio',
 'from_ratio']

In [4]:

df.describe()


,poi,bonus,exercised_stock_options,expenses,long_term_incentive,other,restricted_stock,salary,shared_receipt_with_poi,total_payments,total_stock_value,to_ratio,from_ratio
count,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000
mean,0.124138,7.574308,9.661799,6.729323,5.852428,6.318502,9.831722,8.007985,3.728530,11.544968,12.004710,0.306178,0.091930
std,0.330882,6.790419,6.536471,5.088038,6.546970,5.458204,5.861261,5.961404,3.344199,5.101617,5.128449,0.519126,0.147079
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.419647,12.306560,0.000000,0.000000
50%,0.000000,12.611541,13.317664,9.843472,0.000000,6.854355,12.795328,12.257246,4.744932,13.727988,13.770381,0.020502,0.000000
75%,0.000000,13.592368,14.327292,10.880365,12.832941,11.921446,13.457293,12.502753,6.803505,14.475287,14.640900,0.454255,0.181017
max,1.000000,15.894952,17.352066,12.340446,15.453620,16.153437,16.507546,13.921004,8.616495,18.455660,17.709575,2.527632,0.693147


#### 1.Summarize for us the goal of this project and how machine learning is useful in trying to accomplish it. As part of your answer, give some background on the dataset and how it can be used to answer the project question. Were there any outliers in the data when you got it, and how did you handle those?  [relevant rubric items: “data exploration”, “outlier investigation”]
The goal of this project is to identify the POI from the given data, machine learning is able to dig into the data and extract some useful information. The dataset mainly contain two part of data, financial features: salary, bonus , etc and email features: email_address, from_message, to message etc.

There is a data called"TOTAL" which is an outlier that sum up all other data, I have removed it since the beginning of the analysis. There are also quite a lot missing data in the dataset. I have remove the features which have NaN larger than a threshold, I have try to remove more features but turn out I keep my first try as it give the greatest performence. 

In [12]:
##total number of data points
print "Number of data point :" ,len(data_dict)
##allocation across classes (POI/non-POI)

count_poi=0
for name in data_dict:
    count_poi=count_poi+data_dict[name]['poi']  ## since poi is either 1 or 0, sum of it is the total numnber of poi
count_non_poi=len(data_dict)-count_poi
print "Number of POI : ", count_poi
print "Number of non_POI : ", count_non_poi
print "This number of POI and non-POI is before removing outlier TOTAL"

##number of features used
print"Number of features used :", len(features_list)-1 ### poi is not included for features used.
##are there features with many missing values? etc.
print"========"*8
print"Number of Missing data for each features:"
df.isnull().sum()

Number of data point : 146
Number of POI :  18
Number of non_POI :  128
This number of POI and non-POI is before removing outlier TOTAL
Number of features used : 12
Number of Missing data for each features:


salary                        51
to_messages                   59
deferral_payments            107
total_payments                21
exercised_stock_options       44
bonus                         64
restricted_stock              36
shared_receipt_with_poi       59
restricted_stock_deferred    128
total_stock_value             20
expenses                      51
loan_advances                142
from_messages                 59
other                         53
from_this_person_to_poi       59
poi                            0
director_fees                129
deferred_income               97
long_term_incentive           80
email_address                  0
from_poi_to_this_person       59
dtype: int64

In [14]:
drop_list=df.isnull().sum().index.values[ np.array(df.isnull().sum() > 80) ]
print drop_list
### Please note that the features which have more than 80 missing data are removed and the dropped featured are listed.

['deferral_payments' 'restricted_stock_deferred' 'loan_advances'
 'director_fees' 'deferred_income']


#### 2. What features did you end up using in your POI identifier, and what selection process did you use to pick them? Did you have to do any scaling? Why or why not? As part of the assignment, you should attempt to engineer your own feature that does not come ready-made in the dataset -- explain what feature you tried to make, and the rationale behind it. (You do not necessarily have to use it in the final analysis, only engineer and test it.) In your feature selection step, if you used an algorithm like a decision tree, please also give the feature importances of the features that you use, and if you used an automated feature selection function like SelectKBest, please report the feature scores and reasons for your choice of parameter values.  [relevant rubric items: “create new features”, “properly scale features”, “intelligently select feature”]

The features I kept is shown below, I have do MinMaxScaler scaling as I want to perform PCA and the unit for different features is different, it will be dominant by some of the features if I don't scale it first.
I have create two features, to_ratio and from_ratio , which is correspond to from_poi_to_this_person/from_messages and from_this_person_to_poi/to_messages. The reason behind is that I think for the potential POI, they will have more communication within their community than other normal people. A single measurement from POI or to POI may have bias as certain people may really need to communicate to them more frequently. Thus, I scale it to the from_messages and to_messages which imples not the absolute amount of messages, but the portion of their message to/from poi.
I have use pipeline and gridsearchCV to tune the parameter, all features excepted the dropped features is pass to the pipepline.
There are 4 steps in the pipeline:
1. MinMaxScaler
2. SelectKBest
3. PCA
4. Classifier
The optimized parameter is selected, which the 8 best features are selected by the SelectKBest and the first 4 PCs is choosed by PCA. 


In [5]:

print"Selected Features:","==="*20
selected=clf.named_steps['selection'].get_support()
features_list.remove('poi')
np.array(features_list)[selected]

Selected Features: ============================================================


array(['bonus', 'expenses', 'other', 'salary', 'shared_receipt_with_poi',
       'total_payments', 'to_ratio', 'from_ratio'], 
      dtype='|S23')

In [10]:
clf.named_steps['selection'].scores_

array([ 10.31315372,   0.02858096,  12.93526457,   3.29949313,
        16.87507603,   5.39384063,   8.84385607,   5.13845767,
         6.06693833,   5.44001362,  12.04371759,  16.0935593 ])

#### summary of the performence on the algorithm with "to_ratio" and "from_ratio" added.
Accuracy: 0.84580	Precision: 0.40440	Recall: 0.33100	F1: 0.36404	F2: 0.34347
	Total predictions: 15000	True positives:  662	False positives:  975	False negatives: 1338	True negatives: 12025
#### without the new features added
Accuracy: 0.82340	Precision: 0.30878	Recall: 0.26200	F1: 0.28347	F2: 0.27019
	Total predictions: 15000	True positives:  524	False positives: 1173	False negatives: 1476	True negatives: 11827

#### 3. What algorithm did you end up using? What other one(s) did you try? How did model performance differ between algorithms?  [relevant rubric item: “pick an algorithm”]
I use SelectKBest then PCA and AdaBoostClassifier. I have also try GradientBoosting GaussianNB, SVC. I find Adaboost give me the highest performance then I keep it as my choice. The performence is shown below.

Supplimentary answer: I should explain this part more, for the highest performance I mentioned, I mean the training classifier performence not the test classifier. The procedures I take is 1. run different algorithm with different parameter using Pipeline and GridsearchCV . 2. pick the classifier which give me the highest performance in the training set. 3. Add some more parameter to test the classifier I choosed( As if i try too many parameter for each classifier it takes me too much time, so I just test with less parameter choice at first) 4. Build the final classifier based on the chose classifier.

Please correct me if I am wrong, I just want to make sure my concept is right.
When we are building a classifier, we SHOULD train the classifier with train/test split. A furhter train/cross-validation split in the training set is done and we should choose the classifier works best on which have the best performence in the cross-validation test. The classifier then pass to the testing set to get the final performance.


In [ ]:
### AdaBoostClassifier
    precision    recall  f1-score   support

      False       0.95      0.93      0.94        40
       True       0.40      0.50      0.44         4

avg / total       0.90      0.89      0.89        44

In [ ]:
  ####SVC
    precision    recall  f1-score   support

      False       0.92      0.88      0.90        40
       True       0.17      0.25      0.20         4

avg / total       0.85      0.82      0.83        44

In [ ]:
###GaussianNB
       precision    recall  f1-score   support

      False       0.91      0.97      0.94        40
       True       0.00      0.00      0.00         4

avg / total       0.82      0.89      0.85        44

### I am very confused on why it can not predict a single true POI, as I have seen people using this algorithm on the discussion
### forum, I did some work to remove some features and try this algorithm seperately but still very poor performence, the 
### process is not recorded properly as I have just did it in the console and lost the record after. Any insight why this 
### classifier work so bad?

In [ ]:
### AdaBoostClassifier
  precision    recall  f1-score   support

        0.0       0.97      0.93      0.95        40
        1.0       0.50      0.75      0.60         4

avg / total       0.93      0.91      0.92        44
## As the performence is the best among the classifiers, therefore I choose it.

#### What does it mean to tune the parameters of an algorithm, and what can happen if you don’t do this well?  How did you tune the parameters of your particular algorithm? (Some algorithms do not have parameters that you need to tune -- if this is the case for the one you picked, identify and briefly explain how you would have done it for the model that was not your final choice or a different model that does utilize parameter tuning, e.g. a decision tree classifier).  [relevant rubric item: “tune the algorithm”]
Tuning the parameters is analogous to tune a TV cable, you may fail to watch the tv channel if you do not do so, or get bad quality. It is important to fine tune the classifier to maximize the performence. I use GridSearchCV did a lot of testing on max depth, imputer, learning rate, pca  and selectedKbest features.

#### What is validation, and what’s a classic mistake you can make if you do it wrong? How did you validate your analysis?  [relevant rubric item: “validation strategy”]
Classic mistake is overfitting the model, if a cross-validation test is not used, you can always get a classifier that have 100% accuracy, as it is basically doing an exam with answer provided. 

First,70% data is split as training data and 30% data is testing data. Then a Stratifiedshufflesplit it used to split the training set to training and testing set(cross-validation set), the testing set is the cross validation set that the gridsearchCV selected the best parameter base on data train and predict in the validation set.
The best estimator is then used to submit to the grader for final testing.

StratifiedShuffleSplit is used because the dataset is relatively small and POI is only accounted for a small number. A StratifiedShuffleSplit is to make sure the training set have POI in it by keeping the percentage of class as otherwise it can rate bad classifier which always classify Non-POI as a good classifier.

If no cross-validation is done, you can always overfitting by trial and error and you eventually get a classifier that can predict accurately on the particular testing set but which is very likely not working to future data.

For recall and precision, I take the performence of my selected classifier as an example.
Precision and recall both take the range [0 1].
Precision = True positive/ ( True Positive + False Positive)
Recall = True positive/ ( True Positive + False Negative)

For precision, it is a summary on the classifier where the proportion of True classification is right. In the example, precision =0.402 means  when 100 True classifications is made, 40.2 is positive true classification.

For recall, it is a summary on the classifier where the proportion of True classification is predicted by the classifier. in th eexample, recall = 0.331. It mean when there are 100 True labels, 33.1 of them will be picked up by the classifier.

A high precision indicate that whenever the classifier make a True classification, it is very likely to be correct.
A high recall indicate that the classifier is able to classify most of the positive True labels from the data.

In [ ]:
### AdaBoostClassifier
Accuracy: 0.84527	Precision: 0.40243	Recall: 0.33100	F1: 0.36324	F2: 0.34318
	Total predictions: 15000	True positives:  662	False positives:  983	False negatives: 1338	True negatives: 12017
